In [3]:
# --- 파일명: 1_extract_full_latents.py (수정됨) ---

import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from tqdm.auto import tqdm
from PIL import Image # PIL 라이브러리 추가

# --- 기존 코드에서 모델/데이터셋 클래스 정의 가져오기 ---
class ResBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = torch.nn.Sequential(torch.nn.Conv2d(in_channels, out_channels, 3, 1, 1), torch.nn.BatchNorm2d(out_channels), torch.nn.SiLU(), torch.nn.Conv2d(out_channels, out_channels, 3, 1, 1), torch.nn.BatchNorm2d(out_channels))
        self.shortcut = torch.nn.Conv2d(in_channels, out_channels, 1) if in_channels != out_channels else torch.nn.Identity()
        self.silu = torch.nn.SiLU()
    def forward(self, x): return self.silu(self.block(x) + self.shortcut(x))

class EncoderSuperDeep(torch.nn.Module):
    def __init__(self, in_channels=3, base_channels=128, latent_channels=1):
        super().__init__()
        self.encoder = torch.nn.Sequential(torch.nn.Conv2d(in_channels, base_channels, 3, 1, 1), ResBlock(base_channels, base_channels), torch.nn.Conv2d(base_channels, base_channels*2, 3, 2, 1), ResBlock(base_channels*2, base_channels*2), torch.nn.Conv2d(base_channels*2, base_channels*4, 3, 2, 1), ResBlock(base_channels*4, base_channels*4), torch.nn.Conv2d(base_channels*4, base_channels*8, 3, 2, 1), ResBlock(base_channels*8, base_channels*8), torch.nn.Conv2d(base_channels*8, base_channels*16, 3, 2, 1), ResBlock(base_channels*16, base_channels*16), torch.nn.Conv2d(base_channels*16, 2 * latent_channels, 3, 1, 1))
    def forward(self, x):
        x = self.encoder(x)
        mu, log_var = torch.chunk(x, 2, dim=1)
        return mu, log_var
        
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.paths = sorted([os.path.join(root_dir, f) for f in os.listdir(root_dir) if f.lower().endswith(('.jpg','.jpeg','.png'))])
        self.transform = transform
    def __len__(self): return len(self.paths)
    
    # --- 수정된 __getitem__ 메소드 ---
    def __getitem__(self, idx):
        # torch.load() 대신 Image.open()을 사용해야 합니다.
        img = Image.open(self.paths[idx]).convert('RGB') 
        if self.transform:
            img = self.transform(img)
        return img, self.paths[idx]

def extract_and_save_latents():
    print("--- 1단계: 전체 잠재 벡터 추출 및 저장 시작 ---")
    DEVICE = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
    DATA_DIR = '/home/nas/data/YMG/datas/celeba_hq_256/'
    AE_CHECKPOINT_DIR = '/home/nas/data/YMG/superdeep_ae/checkpoints/'
    ENCODER_PATH = os.path.join(AE_CHECKPOINT_DIR, 'encoder_superdeep_best.pth')
    
    OUTPUT_DIR = '/home/nas/data/YMG/superdeep_ae/my_checkpoints/'
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    LATENT_VECTORS_PATH = os.path.join(OUTPUT_DIR, 'real_latent_vectors_full.pt')

    encoder = EncoderSuperDeep().to(DEVICE)
    encoder.load_state_dict(torch.load(ENCODER_PATH, map_location=DEVICE))
    encoder.eval()

    transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor(), transforms.Normalize(mean=[0.5], std=[0.5])])
    full_dataset = CustomImageDataset(DATA_DIR, transform)
    # CelebA-HQ는 30,000개 이므로 전체 데이터를 사용
    loader = DataLoader(full_dataset, batch_size=128, shuffle=False, num_workers=4)

    all_mu_vectors = []
    with torch.no_grad():
        for images, _ in tqdm(loader, desc="전체 잠재 벡터 추출 중"):
            images = images.to(DEVICE)
            mu, _ = encoder(images)
            all_mu_vectors.append(mu.view(mu.size(0), -1).cpu())
    
    latent_vectors = torch.cat(all_mu_vectors, dim=0)
    
    # 잠재 벡터의 차원을 확인하기 위해 flatten 후의 shape를 사용
    latent_dim = latent_vectors.shape[1]
    torch.save(latent_vectors, LATENT_VECTORS_PATH)
    print(f"\n총 {latent_vectors.shape[0]}개의 잠재 벡터 추출 완료 (차원: {latent_dim})")
    print(f"저장 완료! -> {LATENT_VECTORS_PATH}")

if __name__ == '__main__':
    extract_and_save_latents()

--- 1단계: 전체 잠재 벡터 추출 및 저장 시작 ---


/tmp/ipykernel_2737016/1884837042.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder.load_state_dict(torch.load(ENCODER_PATH, map_location=DEVICE))


전체 잠재 벡터 추출 중:   0%|          | 0/235 [00:00<?, ?it/s]


총 30000개의 잠재 벡터 추출 완료 (차원: 256)
저장 완료! -> /home/nas/data/YMG/superdeep_ae/my_checkpoints/real_latent_vectors_full.pt


In [7]:
# --- 파일명: 2_train_gmm_candidates.py (메모리 최적화 버전) ---

import os
import torch
import joblib
from tqdm.auto import tqdm

# ================================================================
# ✨✨✨ 메모리 사용량 제어 ✨✨✨
# ================================================================
# 이 값을 줄이면 메모리 사용량이 줄어듭니다. (GPU 사양에 맞게 조절)
BATCH_SIZE = 2048
# ================================================================

# --- PyTorchGMM 클래스 정의 (미니배치 fit 메소드로 수정) ---
class PyTorchGMM:
    def __init__(self, n_components, max_iter=100, tol=1e-4, random_state=42):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol
        self.random_state = random_state
        self.weights_ = None
        self.means_ = None
        self.covariances_ = None

    def fit(self, X):
        torch.manual_seed(self.random_state)
        n_samples, n_features = X.shape
        self.device = X.device

        # 초기화
        initial_indices = torch.randperm(n_samples)[:self.n_components]
        self.means_ = X[initial_indices]
        self.covariances_ = torch.eye(n_features, device=self.device, dtype=X.dtype).unsqueeze(0).repeat(self.n_components, 1, 1) * 0.1
        self.weights_ = torch.ones(self.n_components, device=self.device, dtype=X.dtype) / self.n_components
        
        prev_log_likelihood = -torch.inf

        for i in range(self.max_iter):
            tqdm_desc = f'GMM (K={self.n_components}) Iter {i+1}/{self.max_iter}'
            
            # --- M-step을 위한 통계량 누적 변수 초기화 ---
            nk_accumulator = torch.zeros(self.n_components, device=self.device, dtype=X.dtype)
            means_numerator = torch.zeros_like(self.means_, dtype=X.dtype)
            cov_numerator = torch.zeros_like(self.covariances_, dtype=X.dtype)
            total_log_likelihood = 0

            # --- 미니배치 루프 ---
            for batch_X in tqdm(torch.split(X, BATCH_SIZE), desc=tqdm_desc, leave=False):
                # E-step (for batch)
                log_resp = self._estimate_log_prob(batch_X) + torch.log(self.weights_)
                log_prob_norm = torch.logsumexp(log_resp, dim=1, keepdim=True)
                total_log_likelihood += log_prob_norm.sum()
                resp = torch.exp(log_resp - log_prob_norm)

                # M-step 통계량 누적
                nk_batch = resp.sum(dim=0)
                nk_accumulator += nk_batch
                means_numerator += torch.matmul(resp.T, batch_X)

                diff = batch_X.unsqueeze(1) - self.means_.unsqueeze(0)
                outer_prod = diff.unsqueeze(3) @ diff.unsqueeze(2)
                cov_numerator += torch.sum(resp.unsqueeze(2).unsqueeze(3) * outer_prod, dim=0)

            # M-step (전체 데이터에 대한 통계량으로 파라미터 업데이트)
            self.weights_ = nk_accumulator / n_samples
            self.means_ = means_numerator / nk_accumulator.unsqueeze(1)
            self.covariances_ = cov_numerator / nk_accumulator.view(-1, 1, 1)
            self.covariances_ += torch.eye(n_features, device=self.device, dtype=X.dtype) * 1e-6

            # 수렴 확인
            if torch.abs(total_log_likelihood - prev_log_likelihood) < self.tol and i > 0:
                print(f"Iteration {i+1}에서 수렴 완료.")
                break
            prev_log_likelihood = total_log_likelihood
            
    def _estimate_log_prob(self, X):
        # ... (이전 코드와 동일) ...
        # (코드가 길어져 생략, 실제 실행 시에는 포함)
        log_probs = []
        for k in range(self.n_components):
            try:
                cholesky = torch.linalg.cholesky(self.covariances_[k])
                dist = torch.distributions.MultivariateNormal(self.means_[k], scale_tril=cholesky)
                log_probs.append(dist.log_prob(X))
            except torch.linalg.LinAlgError:
                log_probs.append(torch.full((X.shape[0],), -torch.inf, device=self.device, dtype=X.dtype))
        return torch.stack(log_probs, dim=1)

def train_gmms():
    print("--- 2단계: GMM 후보군 학습 시작 (메모리 최적화 버전) ---")
    DEVICE = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
    CHECKPOINT_DIR = '/home/nas/data/YMG/superdeep_ae/my_checkpoints/'
    LATENT_VECTORS_PATH = os.path.join(CHECKPOINT_DIR, 'real_latent_vectors_full.pt')
    
    latent_vectors = torch.load(LATENT_VECTORS_PATH).to(DEVICE)
    latent_vectors = latent_vectors.to(torch.float64) 

    k_candidates = [10, 20, 40, 80, 160, 320]

    for k in k_candidates:
        gmm = PyTorchGMM(n_components=k, random_state=42)
        gmm.fit(latent_vectors)
        
        gmm_params = {
            'weights': g.weights_.cpu().numpy(),
            'means': gmm.means_.cpu().numpy(),
            'covariances': gmm.covariances_.cpu().numpy()
        }
        
        model_path = os.path.join(CHECKPOINT_DIR, f'gmm_k_{k}.pkl')
        joblib.dump(gmm_params, model_path)
        print(f"K={k} GMM 학습 완료 및 저장: {model_path}\n")

if __name__ == '__main__':
    train_gmms()

--- 2단계: GMM 후보군 학습 시작 (메모리 최적화 버전) ---


/tmp/ipykernel_2737016/4219987282.py:96: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  latent_vectors = torch.load(LATENT_VECTORS_PATH).to(DEVICE)


GMM (K=10) Iter 1/100:   0%|          | 0/15 [00:00<?, ?it/s]

GMM (K=10) Iter 2/100:   0%|          | 0/15 [00:00<?, ?it/s]

GMM (K=10) Iter 3/100:   0%|          | 0/15 [00:00<?, ?it/s]

...

GMM (K=10) Iter 97/100:   0%|          | 0/15 [00:00<?, ?it/s]

GMM (K=10) Iter 98/100:   0%|          | 0/15 [00:00<?, ?it/s]

GMM (K=10) Iter 99/100:   0%|          | 0/15 [00:00<?, ?it/s]

GMM (K=10) Iter 100/100:   0%|          | 0/15 [00:00<?, ?it/s]

NameError: name 'g' is not defined

In [ ]:
# --- 파일명: 3_evaluate_fid.py ---

import os
import torch
import joblib
import numpy as np
from torchvision.utils import save_image
from tqdm.auto import tqdm
import torch.distributions as D

# --- DecoderSuperDeep, ResBlock, denormalize 정의 필요 ---
# (이전 코드와 동일하므로 생략, 실제 실행 시에는 포함해야 함)
# ...

def calculate_fid_scores():
    print("--- 3단계: FID 점수 계산 시작 ---")
    DEVICE = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
    REAL_DATA_PATH = '/home/nas/data/YMG/datas/celeba_hq_256/'
    CHECKPOINT_DIR = '/home/nas/data/YMG/superdeep_ae/my_checkpoints/'
    DECODER_PATH = '/home/nas/data/YMG/superdeep_ae/checkpoints/decoder_superdeep_best.pth'
    GEN_IMAGES_ROOT = '/home/nas/data/YMG/superdeep_ae/generated_images_for_fid/'
    
    N_SAMPLES = 10000 # FID 계산을 위한 샘플 수 (시간 관계상 10k로, 권장은 50k)
    BATCH_SIZE = 128
    
    decoder = DecoderSuperDeep().to(DEVICE); decoder.eval()
    decoder.load_state_dict(torch.load(DECODER_PATH, map_location=DEVICE))

    k_candidates = [10, 20, 40, 80, 160, 320]
    fid_results = {}

    for k in k_candidates:
        print(f"\n--- K={k} FID 평가 시작 ---")
        # 1. GMM 모델 로드
        model_path = os.path.join(CHECKPOINT_DIR, f'gmm_k_{k}.pkl')
        gmm_params = joblib.load(model_path)
        weights = torch.from_numpy(gmm_params['weights']).float()
        means = torch.from_numpy(gmm_params['means']).float()
        covs = torch.from_numpy(gmm_params['covariances']).float()

        # 2. 이미지 생성 및 저장
        gen_path_k = os.path.join(GEN_IMAGES_ROOT, f'k_{k}')
        os.makedirs(gen_path_k, exist_ok=True)
        
        print(f"{N_SAMPLES}개 이미지 생성 중...")
        with torch.no_grad():
            for i in tqdm(range(0, N_SAMPLES, BATCH_SIZE), leave=False):
                # GMM에서 샘플링 (MixtureSameFamily 사용)
                mix = D.Categorical(weights)
                comp = D.MultivariateNormal(means, covs)
                gmm_dist = D.MixtureSameFamily(mix, comp)
                z_samples = gmm_dist.sample((BATCH_SIZE,)).to(DEVICE)
                
                generated_images = decoder(z_samples.view(BATCH_SIZE, 1, 16, 16))
                generated_images = (generated_images.clamp(-1, 1) + 1) / 2 # Denormalize
                
                for j, img in enumerate(generated_images):
                    save_image(img, os.path.join(gen_path_k, f'img_{i+j}.png'))

        # 3. FID 점수 계산
        print("FID 점수 계산 중...")
        # pytorch-fid 라이브러리를 시스템 명령어로 호출
        command = f'python -m pytorch_fid "{REAL_DATA_PATH}" "{gen_path_k}" --device {DEVICE}'
        
        # os.system(command)을 사용하거나 아래처럼 결과값을 가져올 수 있음
        import subprocess
        result = subprocess.run(command, shell=True, capture_output=True, text=True)
        try:
            fid_score = float(result.stdout.strip().split(' ')[-1])
            fid_results[k] = fid_score
            print(f"K={k} 의 FID 점수: {fid_score:.4f}")
        except (ValueError, IndexError):
            print("FID 점수를 파싱하는 데 실패했습니다.")
            print("STDOUT:", result.stdout)
            print("STDERR:", result.stderr)

    print("\n--- 최종 FID 점수 요약 ---")
    for k, score in fid_results.items():
        print(f"K = {k:3d} | FID = {score:.4f}")

if __name__ == '__main__':
    calculate_fid_scores()